# Uncapacitated Facility Location (UFL) Notebook
This notebook contains the code for running the UFL problem using Python and Gurobi. To properly use this notebook please ensure you can run ipython notebooks (using Jupyter or other notebook deployments, note this is not the same as running .py files), and have Gurobi solver properly installed and set up with your Python deployment. Please run each cell sequentially, to run a cell select it and click Ctrl+Enter. 

In [1]:
import numpy as np
from numpy import random
from gurobipy import *

In [2]:
# Initialize Model Parameters

M = 3 # number of customers
N = 3 # number of facilities
fixedcost = 15.0 # fixed cost of opening a facility
Min = 0.0 # lower bound for uniform distribution
Max = 40.0 # upper bound for uniform distibution

#Set seed for random parameters
random.seed(0)

# varcost = np.floor((Max - Min) * random.rand(N,M) + Min) # variable cost
# demand = np.floor((Max - Min) * random.rand(M) + Min) # demand of customers
varcost = np.array([[10,1,1],[1,10,1],[1,1,10]])

In [3]:
# Initialize Model
ufl = Model('ufl')

# Define Variables
FracMet = {} # variables corresponding to fraction of demand met
Open = {} # indicator variables for opening facility

for ind_i in range(N):
    Open[ind_i] = ufl.addVar(lb=0.0, ub=1.0, vtype=GRB.BINARY, name='Open'+str(ind_i))
    for ind_j in range(M):
        FracMet[ind_i, ind_j] = ufl.addVar(lb=0.0, ub=1.0, name='FracMet'+str((ind_i, ind_j)))

# Define Objective
ufl.setObjective(quicksum(Open[ind_i]*fixedcost for ind_i in Open.keys()) 
                 + quicksum(FracMet[ind_i,ind_j]*varcost[ind_i, ind_j] for ind_i, ind_j in FracMet.keys()))

# Define Constraints
MeetDemand = {} # meet demand constraitns
IfOpen = {} # indicator constraints

for ind_j in range(M):
    MeetDemand[ind_j] = ufl.addConstr(quicksum(FracMet[ind_i, ind_j] for ind_i in range(N)), '=', 1)
    
for ind_i in range(N):
#     =========================== Aggregated Constraints=====================================
    IfOpen[ind_i] = ufl.addConstr(quicksum(FracMet[ind_i, ind_j] for ind_j in range(M)), '<=', 
                                  M*Open[ind_i], name='IfOpen' + str(ind_i))
# ===========================================================================================
    
#     ============================== Strong Formulation ====================================
#    comment out the above constraints and uncomment these constraints to change from weak to strong formulation
#     -------------------------------------------------------------------------------------
#     for ind_j in range(M):
#         IfOpen[ind_i,ind_j] = ufl.addConstr(FracMet[ind_i,ind_j], '<=', Open[ind_i], name='IfOpen'+str((ind_i,ind_j)))
#     =======================================================================================


#Add constraints and variables to model
ufl.update()

Academic license - for non-commercial use only


In [4]:
# Initialize Model and Solver Settings NOTE: DO NOT EDIT THE SETTINGS IN THIS BLOCK UNLESS OTHERWISE NOTED IN THE EXERCISE
ufl.setParam('TimeLimit', 900)
ufl.setParam('NodefileStart', 100)
ufl.setParam('Presolve', 0)
ufl.setParam('Cuts', 0)
ufl.setParam('Heuristics', 0)
ufl.setParam('BranchDir', -1)
ufl.setParam('VarBranch', 2) # variable selection max infeasibility
ufl.setParam('GomoryPasses', 0) # 0 off, -1 max
ufl.setParam('CliqueCuts', 0) # 0 off, 2 aggressive
ufl.setParam('CoverCuts', 0) # 0 off, 2 aggressive
# ufl.setParam('NodeLimit', 10) # 


ufl.modelSense = GRB.MINIMIZE
ufl.update()



Changed value of parameter TimeLimit to 900.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter NodefileStart to 100.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter Cuts to 0
   Prev: -1  Min: -1  Max: 3  Default: -1
Changed value of parameter Heuristics to 0.0
   Prev: 0.05  Min: 0.0  Max: 1.0  Default: 0.05
Changed value of parameter BranchDir to -1
   Prev: 0  Min: -1  Max: 1  Default: 0
Changed value of parameter VarBranch to 2
   Prev: -1  Min: -1  Max: 3  Default: -1
Changed value of parameter GomoryPasses to 0
   Prev: -1  Min: -1  Max: 2000000000  Default: -1
Changed value of parameter CliqueCuts to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter CoverCuts to 0
   Prev: -1  Min: -1  Max: 2  Default: -1


In [5]:
# Optimize Model (you should see output when running this cell)
ufl.optimize()

Optimize a model with 6 rows, 12 columns and 21 nonzeros
Variable types: 9 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Variable types: 9 continuous, 3 integer (3 binary)

Root relaxation: objective 1.800000e+01, 4 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   18.00000    0    3          -   18.00000      -     -    0s
     0     0   18.00000    0    3          -   18.00000      -     -    0s
     0     2   18.00000    0    3          -   18.00000      -     -    0s
*    3     4               2      27.0000000   23.00000  14.8%   0.7    0s

Explored 7 nodes (8 simplex iterations) in 0.03 seconds
Thread count was 8 (of 8 available processors)

Solution count 1: 27 

Optimal solution found (toleranc

In [6]:
FracMet

{(0, 0): <gurobi.Var FracMet(0, 0) (value 0.0)>,
 (0, 1): <gurobi.Var FracMet(0, 1) (value 0.0)>,
 (0, 2): <gurobi.Var FracMet(0, 2) (value 0.0)>,
 (1, 0): <gurobi.Var FracMet(1, 0) (value 1.0)>,
 (1, 1): <gurobi.Var FracMet(1, 1) (value 1.0)>,
 (1, 2): <gurobi.Var FracMet(1, 2) (value 1.0)>,
 (2, 0): <gurobi.Var FracMet(2, 0) (value 0.0)>,
 (2, 1): <gurobi.Var FracMet(2, 1) (value 0.0)>,
 (2, 2): <gurobi.Var FracMet(2, 2) (value 0.0)>}

In [7]:
Open

{0: <gurobi.Var Open0 (value 0.0)>,
 1: <gurobi.Var Open1 (value 1.0)>,
 2: <gurobi.Var Open2 (value 0.0)>}

In [8]:
v = {}
v[1] = np.array([1,0,0,0,1,0,0,0,1,1,1,1])
len(v)

1

In [9]:
v[1]

array([1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1])